# S03-Segmenting and Patching Slides

## 1. Installing CLAM

It is highly recommanded to use CLAM for slide segmentating and patching, because CLAM implements efficient algorithms and flexible interfaces. 

Now, you should visit https://github.com/mahmoodlab/CLAM and read the related instructions to install CLAM in your server.

## 2. My notes

NOTE: read this section **only if** you want to know the details regarding how CLAM implements tissue segmentating and patching. 

### 2.1 Segmentatation Notes

In CLAM, the segmentation is operated at a best level (e.g., downsampling `x64`).

The values of `a_t` and `a_h` for tissue segmentation should be set regarding: 
- `a_t`: area filter threshold for tissue (positive integer, the minimum size of detected foreground contours to consider, relative to **a reference patch size of 512 x 512 at level 0**, e.g. a value 10 means only detected foreground contours of size greater than 10 512 x 512 sized patches at level 0 will be processed, default: `100`)
- `a_h`: area filter threshold for holes (positive integer, the minimum size of detected holes/cavities in foreground contours to avoid, once again **relative to 512 x 512 sized patches at level 0**, default: `16`)

When segmenting the tissues, the value of filter threshold would be automatically determined in a segmentation level.

When finished segmentation, all coordinates of foreground contours of tissues and holes would be rescaled to the highest level `0`.

### 2.2 Patching Notes

All patches are derived from the rectangle box bounding the foreground contours of tissues.

The most important parameters for patches are `patch_size`, `step_size` and `patch_level`, indicating which size at which level we want to extract. Actually when we run patching, the size and moving step is scaled to level 0 using following code:

```python
# Patch Size
patch_downsample = (int(self.level_downsamples[patch_level][0]), int(self.level_downsamples[patch_level][1]))
ref_patch_size = (patch_size * patch_downsample[0], patch_size * patch_downsample[1])
# Step Size
step_size_x = step_size * patch_downsample[0]
step_size_y = step_size * patch_downsample[1]
```
That is to say, all the patches are calculated and generated at level 0. Thus, the final patch coordinates (restored in `h5` files) are at the highest level 0. Also, the actual patch size is equal to `patch_size * level_downsamples[patch_level]` at the highest level 0.

For efficient computation, it is **recommended** to set `patch_level` to 1 and `patch_size` to 256.

### 2.3 Knowing important parameters of WSIs

To better know and use WSIs, you should get familiar with some important parameters of them.
- `image size`: the image size at the highest magnification.
- `the highest magnification`: It could be seen from the parameter **"MPP"**. A MPP of ~0.25 usually indicates a highest magnification of `40x`, and A MPP of ~0.5 usually indicates a highest magnification of `20x`. 
- `downsample levels`: all available downsample levels in WSIs.

Next, we show an example of accessing these parameters using OpenSlide (https://openslide.org/).

In [1]:
import openslide
wsi_path = '/NAS02/RawData/tcga_rcc/TCGA-KL-8336/TCGA-KL-8336-01Z-00-DX1.bfba9373-efa8-4573-8ee6-8ac961f0b65a.svs'
wsi_object = openslide.open_slide(wsi_path)

`image size`: this WSI is with a size of `127655 * 53444` pixels.

In [2]:
wsi_object.dimensions

(127655, 53444)

`the highest magnification`: `'aperio.MPP': '0.2498'`, which indicates that its highest magnification is `40x`.

In [3]:
wsi_object.properties

<_PropertyMap {'aperio.AppMag': '40', 'aperio.DSR ID': 'ap1251-dsr', 'aperio.Date': '07/13/12', 'aperio.Filename': '285308', 'aperio.Focus Offset': '0.000000', 'aperio.ImageID': '285308', 'aperio.Left': '22.248177', 'aperio.LineAreaXOffset': '0.000000', 'aperio.LineAreaYOffset': '0.000000', 'aperio.LineCameraSkew': '0.000236', 'aperio.MPP': '0.2498', 'aperio.OriginalWidth': '133000', 'aperio.Originalheight': '53544', 'aperio.Parmset': 'COVERSLIP', 'aperio.ScanScope ID': 'IPTH5001', 'aperio.StripeWidth': '1000', 'aperio.Time': '22:51:01', 'aperio.Top': '16.991373', 'aperio.User': '131dd1e8-882f-4bfa-afaf-8d5807a90a47', 'openslide.comment': 'Aperio Image Library v11.0.37\r\n127655x53444 (256x256) JPEG/RGB Q=30;Aperio Image Library v10.0.50\r\n133000x53544 [0,100 127655x53444] (256x256) J2K/YUV16 Q=70|AppMag = 40|StripeWidth = 1000|ScanScope ID = IPTH5001|Filename = 285308|Date = 07/13/12|Time = 22:51:01|User = 131dd1e8-882f-4bfa-afaf-8d5807a90a47|Parmset = COVERSLIP|MPP = 0.2498|Left = 2

`downsample levels`: there are four available downsample levels in this WSI file:
- downsample `1x` at a level `0`: the original image, with a size of 127655 * 53444 pixels.
- downsample `4x` at a level `1`: the rescaled image, with a size of 31913 * 13361 pixels.
- downsample `16x` at a level `2`: the rescaled image, with a size of 7978 * 3340 pixels.
- downsample `32x` at a level `3`: the rescaled image, with a size of 3989 * 1670 pixels.

In [4]:
print("All available downsample levels:", wsi_object.level_downsamples)
print("All available dimension levels:", wsi_object.level_dimensions)

All available downsample levels: (1.0, 4.000047002788833, 16.001037508837925, 32.00207501767585)
All available dimension levels: ((127655, 53444), (31913, 13361), (7978, 3340), (3989, 1670))


## 3. Segmenting and patching tissue regions (foreground)

There are two options for you. The first one, official CLAM, is recommended. The second one, our improved CLAM, would requires extra learning costs.

### 3.1 Using official CLAM

Please refer to https://github.com/mahmoodlab/CLAM for the details of tissue segmentation and patching.

### 3.2 Using an improved version of CLAM

We have improved CLAM in terms of 
- more functionalities: e.g., patch image normlization, image Gaussian bluring, and different architectures for feature extracting.
- adapting to a patient-level organization of WSIs.

A detailed bash script (placed at `./tools/scripts/S03-Seg-Patching.sh`) is as follows:

```bash
#!/bin/bash
set -e

# Sample patches of SIZE x SIZE at LEVEL 
LEVEL=1
SIZE=256

# Path where CLAM is installed
DIR_REPO=./CLAM

# Root path to pathology images 
DIR_READ=/NAS02/RawData/tcga_rcc
DIR_SAVE=/NAS02/ExpData/tcga_rcc

cd ${DIR_REPO}

echo "run seg & patching for all slides"
CUDA_VISIBLE_DEVICES=1 python3 create_patches_fp.py \
    --source ${DIR_READ} \
    --save_dir ${DIR_SAVE}/tiles-l${LEVEL}-s${SIZE} \
    --patch_size ${SIZE} \
    --step_size ${SIZE} \
    --preset tcga.csv \
    --patch_level ${LEVEL} \
    --seg --patch --stitch \
    --no_auto_skip --in_child_dir
```

You could run this script using the following command:
```bash
nohup ./S03-Seg-Patching.sh > S03-Seg-Patching.log 2>&1 &
```

Full running logs could be found in `./tools/scripts/S03-Seg-Patching.log`. 

Next, we check if the number of generated `.h5` files are correct.

In [1]:
import os
import os.path as osp
import pandas as pd

# please use your own gdc samples sheet. Here is an example for illustrating
FILEPATH_TO_GDC_SAMPLE_SHEET = "./docs/gdc_sample_sheet.tsv"

df = pd.read_csv(FILEPATH_TO_GDC_SAMPLE_SHEET, sep='\t')
slide_names = [osp.splitext(df.loc[i, "File Name"].strip())[0] for i in df.index]
print("#Slides:", len(slide_names))
df.head()

#Slides: 940


,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,0596623c-c2c5-4de5-b358-d5393e79120e,TCGA-B3-4103-01Z-00-DX1.76bba2e9-0a6d-460b-8ae...,Biospecimen,Slide Image,TCGA-KIRP,TCGA-B3-4103,TCGA-B3-4103-01Z,Primary Tumor
1,b1b3df18-1fcc-40a1-8610-143f06c9748b,TCGA-AL-3468-01Z-00-DX1.F86A4811-D60C-4845-A7A...,Biospecimen,Slide Image,TCGA-KIRP,TCGA-AL-3468,TCGA-AL-3468-01Z,Primary Tumor
2,e55f0570-5c9e-4676-8b65-380ae02a8d63,TCGA-A4-7997-01Z-00-DX1.aa4e2dd8-fac9-43ae-963...,Biospecimen,Slide Image,TCGA-KIRP,TCGA-A4-7997,TCGA-A4-7997-01Z,Primary Tumor
3,04ea6765-f97b-45a3-9c50-7882b2edf61a,TCGA-HE-A5NF-01Z-00-DX1.74ABE42F-E64E-4550-AD8...,Biospecimen,Slide Image,TCGA-KIRP,TCGA-HE-A5NF,TCGA-HE-A5NF-01Z,Primary Tumor
4,212eed8a-ee10-4149-a5c8-7effb1d4747e,TCGA-EV-5903-01Z-00-DX1.04ef7cdf-b282-4ad3-917...,Biospecimen,Slide Image,TCGA-KIRP,TCGA-EV-5903,TCGA-EV-5903-01Z,Primary Tumor


There are three slides that cannot be processed by CLAM, because they only contain the highest level 0, *i.e.*, the largest image view.

In [2]:
DIR_TO_PATCH_COORD = "/NAS02/ExpData/tcga_rcc/tiles-l1-s256/patches"
generated_patch_filenames = []
for f in os.listdir(DIR_TO_PATCH_COORD):
    if f.endswith(".h5"):
        generated_patch_filenames.append(osp.splitext(f)[0])

filenames_not_processed = []
for s in slide_names:
    if s not in generated_patch_filenames:
        filenames_not_processed.append(s)
        print("The slide {} is not found in generated patch files.".format(s))

The slide TCGA-UZ-A9PQ-01Z-00-DX1.C2CB0E94-2548-4399-BCAB-E4D556D533EF is not found in generated patch files.
The slide TCGA-5P-A9KC-01Z-00-DX1.F3D67C35-111C-4EE6-A5F7-05CF8D01E783 is not found in generated patch files.
The slide TCGA-5P-A9KA-01Z-00-DX1.6F4914E0-AB5D-4D5F-8BF6-FB862AA63A87 is not found in generated patch files.


Here we need to exclude these slides from the table saved for possible classification tasks.

In [3]:
PATH_TO_TABLE = "/NAS02/ExpData/tcga_rcc/table/TCGA_RCC_path_subtype.csv"
PATH_TO_NEW_TABLE = "/NAS02/ExpData/tcga_rcc/table/TCGA_RCC_full_path_subtype.csv"
data_tb = pd.read_csv(PATH_TO_TABLE)
keep_idxs = []
for i in data_tb.index:
    if data_tb.loc[i, "pathology_id"] in filenames_not_processed:
        print("The slide {} is excluded.".format(data_tb.loc[i, "pathology_id"]))
    else:
        keep_idxs.append(i)
data_to_save = data_tb.loc[keep_idxs, :]
data_to_save.to_csv(PATH_TO_NEW_TABLE, index=False)
print("Saved new data in {}".format(PATH_TO_NEW_TABLE))

The slide TCGA-UZ-A9PQ-01Z-00-DX1.C2CB0E94-2548-4399-BCAB-E4D556D533EF is excluded.
The slide TCGA-5P-A9KC-01Z-00-DX1.F3D67C35-111C-4EE6-A5F7-05CF8D01E783 is excluded.
The slide TCGA-5P-A9KA-01Z-00-DX1.6F4914E0-AB5D-4D5F-8BF6-FB862AA63A87 is excluded.
Saved new data in /NAS02/ExpData/tcga_rcc/table/TCGA_RCC_full_path_subtype.csv


### 3.3 An example

The running log of the first WSI is presented as follows:
```text
progress: 0.00, 0/940
processing TCGA-2K-A9WE--TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676.svs
Creating patches for:  TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676 ...
Total number of contours to process:  7
Bounding Box: 3600 58550 2625 3345
Contour Area: 4296064.0
Extracted 6 coordinates
Bounding Box: 145333 30435 3281 11666
Contour Area: 10688768.0
Extracted 20 coordinates
Bounding Box: 5776 864 11121 9986
Contour Area: 66040280.0
Extracted 75 coordinates
Bounding Box: 106932 0 49138 16914
Contour Area: 145224504.0
Extracted 186 coordinates
Bounding Box: 68850 0 77108 92570
Contour Area: 3868769667.0
Extracted 3698 coordinates
Bounding Box: 7600 0 76756 78489
Contour Area: 3373277550.0
Extracted 3272 coordinates
Bounding Box: 0 0 18049 737
Contour Area: 6355584.0
Extracted 17 coordinates
start stitching TCGA-2K-A9WE-01Z-00-DX1.ED8ADE3B-D49B-403B-B4EB-BD11D91DD676
original size: 156086 x 92586
downscaled size for stiching: 9755 x 5786
number of patches: 7274
patch size: 256x256 patch level: 1
ref patch size: (1024, 1024)x(1024, 1024)
downscaled patch size: 64x64
progress: 0/7274 stitched
progress: 728/7274 stitched
progress: 1456/7274 stitched
progress: 2184/7274 stitched
progress: 2912/7274 stitched
progress: 3640/7274 stitched
progress: 4368/7274 stitched
progress: 5096/7274 stitched
progress: 5824/7274 stitched
progress: 6552/7274 stitched
segmentation took 11.164469480514526 seconds
patching took 3.314134359359741 seconds
stitching took 7.7146852016448975 seconds
```

From the log shown above, we can find that
- there are 7274 patches (each with a size of 256 * 256 at the level 1) after segmenting and patching tissues,
- and it costs about 14 seconds for processing the first WSI.

The segmentation (see your `RESULTS_DIRECTORY/tiles-l1-s256/masks/`) and stitching (see your `RESULTS_DIRECTORY/tiles-l1-s256/stitches/`) results of the first WSI are shown as follows:

<img src="./docs/S03-example-seg.png" width="60%" align='left' />

<img src="./docs/S03-example-stitch.png" width="60%" align='left' />

## 4. Results

When finishing the steps above, it is expected that your file structure is as follows:
```txt
RESULTS_DIRECTORY/
	├── masks
    		├── slide_1.png
    		├── slide_2.png
    		└── ...
	├── patches
    		├── slide_1.h5
    		├── slide_2.h5
    		└── ...
	├── stitches
    		├── slide_1.png
    		├── slide_2.png
    		└── ...
	└── process_list_autogen.csv
```

`RESULTS_DIRECTORY` would be `/NAS02/ExpData/tcga_rcc/tiles-l1-s256` in our example.